In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_data=pd.read_csv("salary-train.csv")

In [4]:
train_data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [7]:
test_data=pd.read_csv("salary-test-mini.csv")

In [8]:
test_data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [9]:
X_train_text=train_data["FullDescription"].map(lambda t:t.lower())

In [10]:
X_train_text=X_train_text.replace('[^a-zA-Z0-9]', ' ', regex=True)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
vectorizer=TfidfVectorizer(min_df=5)
X_train_text=vectorizer.fit_transform(X_train_text)

In [13]:
X_test_text=test_data["FullDescription"].map(lambda t:t.lower())

In [14]:
X_test_text=X_test_text.replace('[^a-zA-Z0-9]', ' ', regex=True)

In [15]:
X_test_text=vectorizer.transform(X_test_text)

In [16]:
X_train_category=train_data[["LocationNormalized","ContractTime"]].fillna("nan")

In [17]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
vect=DictVectorizer()

In [19]:
X_train_category=vect.fit_transform(X_train_category.to_dict("records"))

In [20]:
X_test_category=test_data[["LocationNormalized","ContractTime"]].fillna("nan")
X_test_category=vect.transform(X_test_category.to_dict("records"))

In [21]:
from scipy.sparse import hstack

In [22]:
X_train=hstack([X_train_text,X_train_category])
X_test=hstack([X_test_text,X_test_category])

In [23]:
y_train=train_data["SalaryNormalized"].values

In [24]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

In [25]:
CV=5
ridge_regressor=Ridge(alpha=1)
scores=cross_val_score(ridge_regressor,X_train,y_train,scoring="neg_mean_absolute_error",cv=CV)
print(scores.mean())

-7701.92896235


In [29]:
ridge_regressor.fit(X_train,y_train)
predicted=ridge_regressor.predict(X_test)
print("{:0.2f} {:0.2f}".format(predicted[0],predicted[1]))

56566.65 37195.06
